This is a copy of my Pytorch-titanic Notebook but it will apply weight decay to our linear model

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

is_kaggle = "KAGGLE_WORKING_DIR" in os.environ or "/kaggle" in os.getcwd()
print("Running on Kaggle:", is_kaggle)

if is_kaggle:
    data_path = "/kaggle/input/titanic/"
else:
    data_path = os.getcwd() + "/"

In [ ]:
import torch
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

Based on fast.ai chapter 5 we'll now iterate on the numpy-titanic notebook by using pytorch and applying some best practices from that chapter

## Prepare Data set

In [ ]:
df = pd.read_csv(data_path + "train.csv")
df

### Handling na values
For linear regression to work we need numerical values, n/a values are not numerical so we should check if our data set contain them.

In [ ]:
df.isna().sum()

We should avoid removing columns or rows. Even the absence of data can sometimes indicate a pattern.

There are many ways to substitute na_values, the easiest of which is to replace na values with the mode value (the most commonly occuring value). This is a good starting point as usually the method of substituion doesn't have a large impact on our results so the mode is good to get an MVP up and running we can iterate on.

In [ ]:
modes = df.mode().iloc[0]
modes

In [ ]:
df.fillna(modes, inplace=True)
df.isna().sum()

In [ ]:
def substitue_na_with_modes(df: pd.DataFrame) -> pd.DataFrame:
    modes = df.mode().iloc[0]
    return df.fillna(modes)

### Converting Category Data to Binary Categorical Values


We can get view our non-numeric or numberic data using the describe function.


In [ ]:
df.describe(include=[object])

Sex and Embarked only have 2, and 3 unique values respectively. It's safe to say these are categorical values.

We should also check if any of our numbers are categorical

In [ ]:
df.describe(include=[np.number])

We can see from its quarile values that PClass is likely also categorical despite being numeric as its only values are 1, 2 or 3. We can confirm this by looking at the [data dictionary](https://www.kaggle.com/competitions/titanic/data) for the kaggle competition and by via pandas.


In [ ]:
df.Pclass.unique()


Sex, the Passenger class and Embarking city are not measurable attributes so we should convert them to Boolean numbers that can be used as co-efficients. In the previous notebook we did this manually however this pandas can do this for us using `Dataframe.get_dummies()`

In [ ]:
categorical_feature_names = ['Sex', 'Embarked', 'Pclass']
df = pd.get_dummies(df, columns=categorical_feature_names, dtype=int)
df.columns

In [ ]:
dummy_column_names = ['Sex_male', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q',
       'Embarked_S']
df[dummy_column_names].head()

In [ ]:
def convert_categories_to_binary_values(df: pd.DataFrame) -> pd.DataFrame:
    categorical_feature_names = ['Sex', 'Embarked', 'Pclass']
    return pd.get_dummies(df, columns=categorical_feature_names, dtype=int)

### Handling long-tail numerical data

In [ ]:
import matplotlib
df.Fare.hist()

The `Fare` column has lots of small values with the occasional very large value. Uniform normalization using the max value isn't ideal when we're dealing with lots of small values with occasional very large values as the variation between the lower numbers will be lost. To normalize the values we can use a log function (log10 here) to bring the numbers down to reasonable ranges. We must use `log10(x+1)` to avoid 0 values as `log10(0)` would give us infinity.

In [ ]:
import math
df['LogFare'] = np.log(df['Fare'] + 1)
df.LogFare.hist()

## Linear Regression

### Pytorch Tensors
For our gradient descent we'll be using Pytorch rather than numpy for this workbook as it will do a lot of the heavy lifting for us. Alongside Tensorflow pytorch is the most commonly used framework for machine learning.

We'll start by creating Tensors for our target values (known survival status) and features (our numerical data).

In [ ]:
from torch import tensor
target_tensor = tensor(df.Survived)
target_tensor

In [ ]:
feature_names = ['Age', 'SibSp', 'Parch', 'LogFare'] + dummy_column_names
feature_df = df[feature_names]
feature_df

In [ ]:
features = feature_df.values
feature_tensor = tensor(features, dtype=torch.float)
feature_tensor

### Normalization
Once all our features are numerical we need to ensure they're somewhat uniform. For Linear regression and many other ML methods having some features be much larger than others will disrupt the process. Rather than do this manually we can have Pytorch do this for us.

In [ ]:
max_values, max_indices = feature_tensor.max(dim=0)
max_values

In [ ]:
feature_tensor = feature_tensor / max_values
feature_tensor

#### Broadcasting
`feature_tensor / max_values` is an example of [broadcasting](https://pytorch.org/docs/stable/notes/broadcasting.html). 
`max_values` is a one dimensional vector with shape (12). `feature_tensor` is a 2 dimensional matrix with shape (892,12). Because `max_values` is the same size as one of `feature_tensor`'s it will be applied to all 891 rows of `feature_tensor`

Broadcasting is useful for large datasets. The calculations are optimized and run on a GPU when available.

### Prepare initial linear co-efficient values
For linear regression we'd like a one dimensional vector of coefficients equal to our number of rows. Unlike in previous examples we don't need a constant as our dummy variables effectively act as a constant.

In [ ]:
torch.manual_seed(442)
feature_count = feature_tensor.shape[1]
coefficients = torch.rand(feature_count) - 0.5
coefficients

Generally we don't want to set a manual seed so we can be aware of how stable our data is or isn't. However for the sake of this lesson I'd like to check I'm getting consistent results with the lesson plan.

### Create Predictions
We calculate the linear function of our parameters by multiplied them against our random Coefficients then summing each row of weighted values up to create a prediction for each passenger
Pytorch's broadcasting can once again be used here to simplify things considerably. We'll print it out to check there aren't any weighted values that are significantly oversized.

In [ ]:
weighted_values = feature_tensor * coefficients
weighted_values[:4]

In [ ]:
predictions = weighted_values.sum(dim=1)
predictions[:10]

### Calculate loss
Our loss here is the average difference between our prediction value and whether the passegner survived or not (1 or 0).

In [ ]:
loss = torch.abs(predictions - target_tensor).mean()
loss

In [ ]:
def create_predictions(features: torch.Tensor, coefficients: torch.Tensor) -> torch.Tensor:
    return (coefficients * features).sum(dim=1)

In [ ]:
def calculate_loss(features: torch.Tensor, coefficients: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    predictions = create_predictions(features, coefficients=coefficients)
    return torch.abs(predictions - targets).mean()

### Doing a single Gradient Descent step
Now we want to optimize our loss with gradient descent. This too will be significantly easier using Pytorch as it will calculate the gradient for us.

We must tell pytorch to store the results of each coefficient calculation so we can get the gradients from it later.

In [ ]:
coefficients.requires_grad_()

In [ ]:
loss = calculate_loss(feature_tensor, coefficients=coefficients, targets=target_tensor)
loss

The loss is in a tensor where can ask Pytorch to calculate the gradient by calling `backward()`

In [ ]:
loss.backward()
coefficients.grad

Here we perform one gradient descent step


In [ ]:
loss = calculate_loss(feature_tensor, coefficients=coefficients, targets=target_tensor)
loss.backward
with torch.no_grad():
    assert coefficients.grad is not None
    coefficients.sub_(coefficients.grad * 0.1)
    coefficients.grad.zero_()
    print(calculate_loss(feature_tensor, coefficients=coefficients, targets=target_tensor))

A few points:
1. `torch.no_grad()` is required to ensure the parameter update step is peformed without tracking gradients. We want to track gradients for the forward and backward steps but not when directly modifying the parameters
2. `coefficients.sub_(coefficients.grad * 0.1)` reduces the coefficients by their gradient to the loss. More significant features will be reduced more. 
3. Both `sub_` and `zero_` operations are done in place for memory efficiency and to preserve the tensors memory graph (this is also ensured by `torch.no_grad()` although it's good practice when working with tensors).
4. `coefficients.grad.zero_()` sets our gradients to zero. This is necessary as if we were to do another backpass the new gradients would be added to the old ones.

### Creating a validation set


Before we begin training we need a validation set to compare our training data against.

I've deviated from the [fast.ai kaggle workbook](https://www.kaggle.com/code/jhoward/linear-model-and-neural-net-from-scratch) as they split their validation set using the fastai library to keep things consistent for their next chapter. I'm interested in primarily learning Pytorch so I'm going to split the dataset without the fastai library. However so I can check if my results match fast.ai's I'm going to include their splitter here it will be used if `use_fastai_splitter` is set to `True` so I can check my results are consistent with the fast.ai tutorials.

In [ ]:
from random import Random
from numpy import int64
from fastai.data.transforms import RandomSplitter
from typing import Tuple, List, cast
from fastcore.foundation import L
from torch import Tensor

def split_data_with_fastai(df: pd.DataFrame) -> Tuple[Tensor,Tensor]:
    train_indices, validation_indices = RandomSplitter(seed=42)(df)
    return torch.tensor(train_indices, dtype=torch.int64), torch.tensor(validation_indices, dtype=torch.int64)

First we'll split our data

In [ ]:
use_fastai_splitter = True
total_passengers = feature_tensor.size(0)
training_set_size = int(total_passengers * 0.8)

if use_fastai_splitter:
    train_indices, validation_indices = split_data_with_fastai(df)
else:
    randomized_indices = torch.randperm(total_passengers)
    train_indices = randomized_indices[:training_set_size]
    validation_indices = randomized_indices[training_set_size:]

training_features = feature_tensor[train_indices]
validation_features = feature_tensor[validation_indices]
training_targets = target_tensor[train_indices]
validation_targets = target_tensor[validation_indices]
len(training_features), len(validation_features)

This note book doesn't use Pytorch's `Dataset`s. We'd likely use these in a real project although for this example we're keeping things a bit barer than normal so we can see the process.

We'll add what we've done so far in to functions to make things easier to read and re-usable.

In [ ]:
def update_coefficients(coefficients, learning_rate):
    coefficients.sub_(coefficients.grad * learning_rate)
    coefficients.grad.zero_()

In [ ]:
def one_epoch(coefficients, learning_rate):
    loss = calculate_loss(training_features, coefficients, training_targets)
    loss.backward()
    with torch.no_grad():
        update_coefficients(coefficients, learning_rate=learning_rate)
        
    print(f"{loss:.3f}", end="; ")

In [ ]:
def generate_coefficients(features: torch.Tensor) -> torch.Tensor:
    coefficient_count = features.shape[1]
    coefficients = torch.rand(coefficient_count) - 0.5
    coefficients.requires_grad_()
    return coefficients

Now to train the model

In [ ]:
def train_model(epoch_count=30, learning_rate=0.1):
    coefficients = generate_coefficients(training_features)
    for i in range(epoch_count):
        one_epoch(coefficients, learning_rate=learning_rate)
    return coefficients

In [ ]:
coefficients = train_model(epoch_count=18, learning_rate=0.2)
coefficients

We can see below that our models has optimized our weights to reduce our loss. From this we can see that the model believes

In [ ]:
def show_coeffs(): 
    coeff_array = [coeff.item() for coeff in coefficients]
    coeff_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coeff_array})
    display(coeff_df)
show_coeffs()

### Measuring accuracy

To view our accuracy we'll now use our validation set. We'll create predictions using our newly trained coefficients and see how accurate they are.

In [ ]:
predictions = create_predictions(validation_features, coefficients=coefficients)
predictions[:10]

If our predictions is >0.5 and the passegner surivied we're correct. If the passenger died we want a prediction < 0.5. 0 = died, 1 = survived. This code merely rounds our predictions to whichever of these values is closest

In [ ]:
results = validation_targets.bool() == (predictions>0.5)
results.float().mean()

We're 79% accurate which is pretty good going.

In [ ]:
from torch import Tensor


def calculate_accuracy(coefficients, features: torch.Tensor) -> Tensor:
    predictions = create_predictions(features, coefficients=coefficients)
    results = validation_targets.bool() == (predictions>0.5)
    return results.float().mean()

### Sigmoid
When creating predictions that are between 0 and 1 we can increase our accuracy by using the sigmoid function which moves all our values between 0 and 1 and larger negative or positives values will respectively asymptotically converge towards 0 or 1.

In [ ]:
import sympy
sympy.plot("1/(1+exp(-x))", xlim=(-5,5))

In [ ]:
def create_predictions(features: torch.Tensor, coefficients: torch.Tensor) -> torch.Tensor:
    summed_weighted_values = (coefficients * features).sum(dim=1)
    return torch.sigmoid(summed_weighted_values)


Constricting the range of our predictions within the range of what they can realistically be makes them much easier to optimize. When this is applied to every prediction each epoch should minimize our loss more effectivelyby by eliminating values that are outside the range of what our predictions can realistically be.

This in turn allows us to substantially increase the learning rate as our loss won't be as high or fluctuate as wildly.

In [ ]:
coefficients = train_model(learning_rate=100)
calculate_accuracy(coefficients, features=validation_features)

83% a sharp improvement!

## Weight Decay
Often our loss will go down but our validation loss will begin to increase. This is usually a sign of overfitting. One of those most basic ways to prevent overfitting is weight decay.

We add all the weights squared to our loss. This will hinder our training but helps prevent overfitting by forcing our weights to get smaller. Smaller weights mean less resolution in our models solutions, as demonstrated a solution that fits our training data too closely will over-fit

![overfitting-illustration](overfitting-example.webp)

It will probably be quite difficult to make this model overfit as its quite a simply model and doesn't have many parameters so for now we'll just print out our validation loss alongside our loss so we can at least compare the two side by side. Having your loss improve but your validation loss worsen is usually a sign of overfitting.

In [ ]:
def one_epoch(coefficients, learning_rate):
    loss = calculate_loss(training_features, coefficients, training_targets)
    loss.backward()
    with torch.no_grad():
        update_coefficients(coefficients, learning_rate=learning_rate)
        validaton_loss = calculate_loss(validation_features, coefficients, validation_targets)
        
    print(f"loss: {loss:.3f}, val_loss: {validaton_loss}", end=";\n")

In [ ]:
coefficients = train_model(learning_rate=100)
calculate_accuracy(coefficients, features=validation_features)

## Prepare Submission CSV

### Using a Test set
Before submitting to Kaggle we'll want to test the effectiveness of our data against our test set. 
#### Why not use the Validation set?
This may seem similar to how we used our validation set but there's an important difference. A validation set is used to give us an unbiased evaluation of our model's performance. Unlike a training set which is biased as we're training our model on it. As we develop our model the validation set will indirectly become biased as we iterate on our model to improve the validation sets accuracy. The test set is only ever used once we have finished developing our model so it gives us an accurate assessment of how our model behaves on completely unseen data.

- Training set - Model bias
- Validation Set - Developer bias
- Test set - No bias

### Clean Test Data

In [ ]:
test_df = pd.read_csv(data_path + 'test.csv')
test_df.isna().sum()

We can use the same steps we took for cleaning the training data on our test data. However it's always worth checking if there are any additional na values. Here there's an na value for Fare that needs resolving.

In [ ]:
test_df['Fare'] = test_df.Fare.fillna(0)
test_df = substitue_na_with_modes(test_df)
test_df = convert_categories_to_binary_values(test_df)
test_df["LogFare"] = np.log(test_df['Fare'] + 1)
test_df.isna().sum()

In [ ]:
test_df[feature_names][:5]

In [ ]:
test_features = tensor(test_df[feature_names].values, dtype=torch.float)
test_features = test_features / max_values
test_features[:3]

In [ ]:
feature_tensor

In [ ]:
coefficients

### Create test predictions

In [ ]:
with torch.no_grad():
    test_predictions = create_predictions(test_features, coefficients=coefficients)
test_predictions = (test_predictions > 0.5).int()
test_predictions

### Create Kaggle submission
We can view the sample submission kaggle has given us to see how to format this.

In [ ]:
sample_df = pd.read_csv(data_path + "gender_submission.csv")
sample_df[:3]

In [ ]:
submission_df = pd.DataFrame({ "PassengerId": test_df["PassengerId"], "Survived": test_predictions })
submission = submission_df.to_csv("submission.csv",index=False)
!head submission.csv

## Neural Nets
The calculation above was a linear regression as we only use one set of parameters in the form of our features which we immedietely derived the output from. This effectively gave us a single layer with one neuron.

Here we'll employ two additional concepts to improve the resolution of our calculations accuracy. We will create a single hidden layer with multiple neurons, apply a RELU (Rectified Linear Unit) function and add them together to give us a loss. A RELU function is non-linear and simply replaces every negative number with a 0.

### Create Matrix of Coefficients

Prior to now we were using a single neuron consisting of a single layer. We're now going to add additional neurons by generating more than one set of coefficients

Our first layer will take our inputs/features and create 20 outputs from 20 different sets of coefficients.

In [ ]:
hidden_layer_neuron_count = 20
layer1 = torch.rand(feature_count, hidden_layer_neuron_count) - 0.5
layer1.shape

When we've more than one neuron we need to adjust the values of each neuron proportionally so we end up with a similar magnitude.

In [ ]:
layer1 = layer1 / hidden_layer_neuron_count
layer1[:3]

### Creating an output layer
The 2nd layer is our output layer which will produce one output predicting the passengers survival. As we're using neurons from a hidden layer as our input and not our features which included dummy values we now need to include a constant (also known as a bias). This 2nd layer consists of one neuron

In [ ]:
layer2 = torch.rand(hidden_layer_neuron_count, 1) - 0.3
constant = torch.rand(1)[0]
layer2.shape

In [ ]:
def generate_coefficients(features: torch.Tensor, hidden_layer_neuron_count: int=20):
    feature_count = features.shape[1]
    layer1 = (torch.rand(feature_count, hidden_layer_neuron_count) - 0.5) / feature_count
    layer2 = torch.rand(hidden_layer_neuron_count, 1) - 0.3
    constant = torch.rand(1)[0]
    return layer1.requires_grad_(), layer2.requires_grad_(), constant.requires_grad_()

### Convert targets to column vectors
Lastly we need to add an additional dimension to our targets so matrix multiplications will work correctly on them.  We need to be very careful to remember to do this as not doing so won't return an error but will give us incorrect results as the Pytorch will attempt to broadcast the mishaped target tensor.

In [ ]:
training_targets = training_targets[:,None]
validation_targets = validation_targets[:,None]

### Matrix Multiplication
Now that we have more than one neuron (more than one row of parameters/coefficients) we need to switch from using pytorch's broadcasting to matrix multiplication so that every row of parameters is multiplied with our features. It's also worth noting using PyTorch's matrix multiplication `@` is very efficient as it's been highly optimized both algorithmically and for operating on multi-core GPUs. You'll see us using this later when we create the neural net.

```python
torch.sigmoid(coefficients @ features)
```

### Updating multiple coefficients
We also need to loop through our coefficients now we have more than one layer. We simpy add a loop to our existing function.

In [ ]:
def update_coefficients(coefficients, learning_rate):
    for layer in coefficients:
        layer.sub_(layer.grad * learning_rate)
        layer.grad.zero_()

### Creating the Neural net
Our plan for this neural net is to have two layers.

12 Features -> |20 neuron hidden layer| --(Relu)-> |1 neuron output layer| --(Sigmoid)-> 1 Survival prediction

In [ ]:
print(df.Name[788])
print(str(feature_tensor[0].shape) + " --> " + str(layer1.shape) + " --> " + str(layer2.shape) + " --> " + str(predictions[0].item()))

If you're curious [Bertram](https://titanic.fandom.com/wiki/Bertram_Vere_Dean) did indeed survive and lived until 1992.


The RELU is needed as adding together two linear functions just gives us another linear function which doesn't give us any more resolution for our calculation. Combining each linear layer with a non-linear RELU allows us to keep each linear functions utility increasing our algortihms accuracy.

In [ ]:
import torch.nn.functional as F

def create_predictions(features: Tensor, coefficients: Tuple[Tensor, Tensor, Tensor]) -> Tensor:
    layer1_parameters, layer2_parameters, constant_parameter = coefficients
    layer1_output = F.relu(features @ layer1_parameters)
    layer2_output = torch.sigmoid(layer1_output @ layer2_parameters + constant_parameter)
    return layer2_output

In [ ]:
coefficients = train_model(learning_rate=20)

In [ ]:
calculate_accuracy(coefficients, features=validation_features)

### Linear Regression vs Neural Net
Our 82.58% accuracy is by (I compared my results against fast.ai's where the same thing happeneed to them) coincidence the same result as we got from the linear model. Usually a neural net will have improved performance against a linear model. However for relatively small simple datasets such as the titanic dataset linear regression will usually do just fine and less complexity isn't neeeded.

## Deep Learning
*Note: This section does not produce the same results as fast.ai or good quality ones for that matter. There's either a mistake in my code or the fine-tuning fast.ai has applied needs me to replicate their data to the letter which isn't the case for this last section. However I'm trying to be less of a perfectionist and I think it still showcases a deep learning model so this should be fine.*

Our neural net consisted of the input layer, one hidden layer and an output layer. For something to really be deep learning we want to have many input layers. In this last section we'll adapt our code so we can have multiple layers we can loop through. For now we'll add one additional layer.

In [ ]:
def generate_coefficients(features: Tensor, hidden_layer_neuron_counts: List[int] = [10,10]) -> Tuple[List[Tensor], List[Tensor]]:
    feature_count = features.shape[1]
    layer_neuron_counts = [feature_count] + hidden_layer_neuron_counts + [1]
    layer_count = len(layer_neuron_counts)
    
    layers = []
    constants = []
    for i in range(layer_count-1):
        current_layer_neuron_count = layer_neuron_counts[i]
        next_layer_neuron_count = layer_neuron_counts[i+1]
        current_layer_weights = torch.rand(current_layer_neuron_count, next_layer_neuron_count) -0.3
        current_layer_weights = current_layer_weights / next_layer_neuron_count * 4
        layers.append(current_layer_weights.requires_grad_())
        current_layer_constant = (torch.rand(1)[0] - 0.5)/next_layer_neuron_count
        constants.append(current_layer_constant.requires_grad_())
    return layers, constants

### Fine Tuning
Note that the above code requires a lot of small fine tunings such as the `- 0.3` deducted from the weight initialization, and the `* 4` applied to the normalization. This is in part why deep learning didn't take off until recently, the networks can be very finnicky to get working. There are however techniques for

The creation of predictions and updating of coefficients also needs to be adjusted for having loops which we do below.

For creating predictions we stop our loop at the last layer so we can apply a sigmoid instead of a relu function for one neuron output.

In [ ]:
def create_predictions(features: Tensor, coefficients: Tuple[List[Tensor], List[Tensor]]) -> Tensor:
    result = features
    layers, constants = coefficients
    for layer, constant in zip(layers[:-1], constants[:-1]):
        result = torch.relu(result @ layer + constant)
    
    return torch.sigmoid(result @ layers[-1] + constants[-1])

In [ ]:
def update_coefficients(coefficients: Tuple[List[Tensor], List[Tensor]], learning_rate: float):
    layers, constants = coefficients
    for layer in layers+constants:
        assert layer.grad is not None
        layer.sub_(layer.grad * learning_rate)
        layer.grad.zero_()

In [ ]:
coefficients = train_model(learning_rate=4)
calculate_accuracy(coefficients, features=validation_features)

The steps for this final deep learning model are rudimentary but do showcase what most deep learning models consist of. There are some key difference between this and more complex modern models.

* Additional techniques for initalization and normalization of parameters
* Regularization (to avoid overfitting)
* Modifying the neural net itself to take advantage of knowledge of the problem domain.
* Using smaller batches for gradient descent rather than putting our entire dataset in at once.

There topics will be addressed in future notebooks.